In [ ]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
!pip install ffmpeg
!pip install git+https://github.com/openai/whisper.git
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=ca09b0221720153913f6fd30facdd5a1b82b590f16751778560bfdc1e49ef64f
  Stored in directory: /root/.cache/pip/wheels/30/33/46/5ab7eca55b9490dddbf3441c68a29535996270ef1ce8b9b6d7
Successfully built ffmpeg
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-0fe1rmsy
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-0fe1rmsy
  Resolved https://github.com/openai/whisper.git to commit 7858aa9c08d98f75575035ecd6481f462d66ca27
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21

In [ ]:
# -*- coding: utf-8 -*-
"""datamodel.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1e7Eas4nZJIig4uxUhGSOzfnj2KVhcAwa
"""

#datacleaning
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import string
import xgboost as xgb
import numpy as np
depression_data=pd.read_csv(r"depression_dataset_reddit_cleaned.csv")
anxiety_data=pd.read_excel(r"anxietydataset.xlsx")
stress_data=pd.read_csv(r"stressdataset.csv")
anxiety_data=anxiety_data.dropna(how='any')
stress_data=stress_data.dropna(how='any')
anxiety_data['text']=anxiety_data['text'].apply(lambda post:re.sub(r"\'"," ",post))
depress_train=depression_data.sample(frac=0.8,random_state=50)
depress_test=depression_data.drop(depress_train.index)
stress_data['text']=stress_data['text'].apply(lambda post:re.sub("\'"," ",post))
contractions_dict = { "ain t ": "are not "," s ":" is ","aren t ": "are not ","can t ": "can not ","can t ve ": "cannot have ",
" cause ": "because ","could ve ": "could have ","couldn t ": "could not ","couldn t ve ": "could not have ",
"didn t ": "did not ","doesn t ": "does not ","don t ": "do not ","hadn t ": "had not ","hadn t ve ": "had not have ",
"hasn t ": "has not ","haven t ": "have not ","he d ": "he would ","he d ve ": "he would have ","he ll ": "he will ",
"he ll ve ": "he will have ","how d ": "how did ","how d y ": "how do you ","how ll ": "how will ","i d ": "i would ",
"i d ve ": "i would have ","i ll ": "i will ","i ll ve ": "i will have ","i m ": "i am ","i ve ": "i have ",
"isn t ": "is not ","it d ": "it would ","it d ve ": "it would have ","it ll ": "it will ","it ll ve ": "it will have ",
"let s ": "let us ","ma am ": "madam ","mayn t ": "may not ","might ve ": "might have ","mightn t ": "might not ",
"mightn t ve ": "might not have ","must ve ": "must have ","mustn t ": "must not ","mustn t ve ": "must not have ",
"needn t ": "need not ","needn t ve ": "need not have ","o clock ": "of the clock ","oughtn t ": "ought not ",
"oughtn t ve ": "ought not have ","shan t ": "shall not ","sha n t ": "shall not ",
"shan t ve ": "shall not have ","she d ": "she would ","she d ve ": "she would have ","she ll ": "she will ",
"she ll ve ": "she will have ","should ve ": "should have ","shouldn t ": "should not ",
"shouldn t ve ": "should not have ","so ve ": "so have ","that d ": "that would ","that d ve ": "that would have ",
"there d ": "there would ","there d ve ": "there would have ",
"they d ": "they would ","they d ve ": "they would have ","they ll ": "they will ","they ll ve ": "they will have ",
"they re ": "they are ","they ve ": "they have ","to ve ": "to have ","wasn t ": "was not ","we d ": "we would ",
"we d ve ": "we would have ","we ll ": "we will ","we ll ve ": "we will have ","we re ": "we are ","we ve ": "we have ",
"weren t ": "were not ","what ll ": "what will ","what ll ve ": "what will have ","what re ": "what are ",
"what ve ": "what have ","when ve ": "when have ","where d ": "where did ",
"where ve ": "where have ","who ll ": "who will ","who ll ve ": "who will have ","who ve ": "who have ",
"why ve ": "why have ","will ve ": "will have ","won t ": "will not ","won t ve ": "will not have ",
"would ve ": "would have ","wouldn t ": "would not ","wouldn t ve ": "would not have ","y all ": "you all ",
"y all d ": "you all would ","y all d ve ": "you all would have ","y all re ": "you all are ","y all ve ": "you all have ",
"you d ": "you would ","you d ve ": "you would have ","you ll ": "you will ","you ll ve ": "you will have ",
"you re ": "you are ","you ve ": "you have ", "aint ": "are not ","arent ": "are not ","cant ": "can not ","cant ve ": "cannot have ",
"couldve ": "could have ","couldnt ": "could not ","couldnt ve ": "could not have ",
"didnt ": "did not ","doesnt ": "does not ","dont ": "do not ","hadnt ": "had not ","hadnt ve ": "had not have ",
"hasnt ": "has not ","havent ": "have not ","hed ": "he would ","hed ve ": "he would have ",
"howd ": "how did ","howd y ": "how do you ","howll ": "how will ","id ": "i would ",
"id ve ": "i would have ","ill ve ": "i will have ","im ": "i am ","ive ": "i have ",
"isnt ": "is not ","itd ": "it would ","itd ve ": "it would have ","itll ": "it will ","itll ve ": "it will have ",
"lets ": "let us ","maam ": "madam ","mightve ": "might have ","mightnt ": "might not ",
"mightnt ve ": "might not have ","mustve ": "must have ","mustnt ": "must not ","mustnt ve ": "must not have ",
"neednt ": "need not ","neednt ve ": "need not have ","oclock ": "of the clock ","oughtnt ": "ought not ",
"oughtnt ve ": "ought not have ","shant ": "shall not ",
"shant ve ": "shall not have ","shed ": "she would ","shed ve ": "she would have ","shell ": "she will ",
"shell ve ": "she will have ","shouldve ": "should have ","shouldnt ": "should not ",
"shouldnt ve ": "should not have ","sove ": "so have ","thatd ": "that would ","thatd ve ": "that would have ",
"thered ": "there would ","thered ve ": "there would have ",
"theyd ": "they would ","theyd ve ": "they would have ","theyll ": "they will ","theyll ve ": "they will have ",
"theyre ": "they are ","theyve ": "they have ","tove ": "to have ","wasnt ": "was not ","wed ": "we would ",
"weve ": "we have ",
"werent ": "were not ","whatll ": "what will ","whatre ": "what are ",
"whatve ": "what have ","whenve ": "when have ","whered ": "where did ",
"whereve ": "where have ","wholl ": "who will ","whove ": "who have ",
"whyve ": "why have ","willve ": "will have ","wont ": "will not ",
"wouldnt ": "would not ","yall ": "you all ",
"yall d ": "you all would ","yall d ve ": "you all would have ","yall re ": "you all are ","yall ve ": "you all have ",
"youd ": "you would ","youd ve ": "you would have ","youll ": "you will ","youll ve ": "you will have ",
"youre ": "you are ","youve ": "you have "}
def contractions(post):  
  for key,value in contractions_dict.items():
    if key in post:
      post=re.sub(key,value,post)
  return post
def cleaning(post):
  post=post.lower()
  post=re.sub(r"http\S+","",post)
  post=re.sub(r"url"," ",post)
  post=re.sub(r"\s[^a-z]\s"," ",post)
  post=re.sub(r"\sop\s"," ",post)
  post=re.sub(r"\s[a-z]\s"," ",post)  
  post=re.sub(r"\s[a-z][a-z]\s"," ",post)
  post=re.sub(r"\w*\d\w*","",post)
  post=re.sub(r"\."," ",post)
  post=re.sub(r"\""," ",post)
  post=re.sub(r"\,"," ",post)
  post=re.sub(r"\(|\)|\[|\]|\{|\}"," ",post)
  post=re.sub(r"\+|\-|\/|\=|\*"," ",post)
  post.translate(str.maketrans('','',string.punctuation))
  post=re.sub(r"\?"," ",post)
  post=re.sub(r"\!"," ",post)
  post=re.sub(r"[^a-z]"," ",post)

  return post
depress_train["clean_text"]=depress_train["clean_text"].apply(lambda post:contractions(post))
depress_test["clean_text"]=depress_test["clean_text"].apply(lambda post:contractions(post))
depress_train["clean_text"]=depress_train["clean_text"].apply(lambda post:cleaning(post))
depress_test["clean_text"]=depress_test["clean_text"].apply(lambda post:cleaning(post))
depress_train["clean_text"]=depress_train["clean_text"].apply(lambda post:re.sub(" +"," ",post))
depress_test["clean_text"]=depress_test["clean_text"].apply(lambda post:re.sub(" +"," ",post))
anxiety_data["text"]=anxiety_data["text"].apply(lambda post:contractions(post))
anxiety_data["text"]=anxiety_data["text"].apply(lambda post:cleaning(post))
anxiety_data["text"]=anxiety_data["text"].apply(lambda post:re.sub(" +"," ",post))
stress_data["text"]=stress_data["text"].apply(lambda post:contractions(post))
stress_data["text"]=stress_data["text"].apply(lambda post:cleaning(post))
stress_data["text"]=stress_data["text"].apply(lambda post:re.sub(" +"," ",post))
anxiety_data.dropna(inplace=True)
depress_train.dropna(inplace=True)
depress_test.dropna(inplace=True)
stress_data.dropna(inplace=True)
stop_words=set(stopwords.words("english"))
depress_train["clean_text"]=[word_tokenize(w) for w in depress_train["clean_text"]]
depress_test["clean_text"]=[word_tokenize(w) for w in depress_test["clean_text"]]
anxiety_data["text"]=[word_tokenize(w) for w in anxiety_data["text"]]
stress_data["text"]=[word_tokenize(w) for w in stress_data["text"]]
cleaned_words=[] 
#removing stopwords
for words in depress_train["clean_text"]:
  temp=[]
  for w in words:
    if w not in stop_words:
      temp.append(w)
  cleaned_words.append(temp)
depress_train["clean_text"]=cleaned_words
cleaned_words=[]
for words in depress_test["clean_text"]:
  temp=[]
  for w in words:
    if w not in stop_words:
      temp.append(w)
  cleaned_words.append(temp)
depress_test["clean_text"]=cleaned_words
cleaned_words=[]
for words in anxiety_data["text"]:
  temp=[]
  for w in words:
    if w not in stop_words:
      temp.append(w)
  cleaned_words.append(temp)
anxiety_data["text"]=cleaned_words
cleaned_words=[]
for words in stress_data["text"]:
  temp=[]
  for w in words:
    if w not in stop_words:
      temp.append(w)
  cleaned_words.append(temp)
stress_data["text"]=cleaned_words      
#lemmatzing by parts of speechh
pos=[]
for words in depress_train["clean_text"]:
  temp=nltk.pos_tag(words)
  pos.append(temp)
allowed_type=["JJ","RB","JJR","JJS","RBR","RBS","NN","NNS","NNP"]
convert={"a":["JJ","JJR","JJS"],"r":["RB","RBR","RBS"],"n":["NN","NNS","NNP"]}
cleaned_words=[]
lematizer=WordNetLemmatizer();
for words in pos:
  temp=[]
  for w in words:
    if w[1] in allowed_type:
      for key,val in convert.items():
        if w[1] in val:
          temp.append(lematizer.lemmatize(w[0],pos=key)) 
  cleaned_words.append(temp)
depress_train["clean_text"]=cleaned_words  

pos=[]
for words in depress_test["clean_text"]:
  temp=nltk.pos_tag(words)
  pos.append(temp)
cleaned_words=[]
lematizer=WordNetLemmatizer();
for words in pos:
  temp=[]
  for w in words:
    if w[1] in allowed_type:
      for key,val in convert.items():
        if w[1] in val:
          temp.append(lematizer.lemmatize(w[0],pos=key)) 
  cleaned_words.append(temp)
depress_test["clean_text"]=cleaned_words  
pos=[]
for words in anxiety_data["text"]:
  temp=nltk.pos_tag(words)
  pos.append(temp)
cleaned_words=[]
lematizer=WordNetLemmatizer();
for words in pos:
  temp=[]
  for w in words:
    if w[1] in allowed_type:
      for key,val in convert.items():
        if w[1] in val:
          temp.append(lematizer.lemmatize(w[0],pos=key)) 
  cleaned_words.append(temp)
anxiety_data["text"]=cleaned_words
pos=[]
for words in stress_data["text"]:
  temp=nltk.pos_tag(words)
  pos.append(temp)
cleaned_words=[]
lematizer=WordNetLemmatizer();
for words in pos:
  temp=[]
  for w in words:
    if w[1] in allowed_type:
      for key,val in convert.items():
        if w[1] in val:
          temp.append(lematizer.lemmatize(w[0],pos=key)) 
  cleaned_words.append(temp)
stress_data["text"]=cleaned_words 
depress_train["clean_text"]=[list(set(t)) for t in depress_train["clean_text"]]
depress_test["clean_text"]=[list(set(t)) for t in depress_test["clean_text"]]
stress_data["text"]=[list(set(t)) for t in stress_data["text"]]
anxiety_data["text"]=[list(set(t)) for t in anxiety_data["text"]]

anxiety_train=anxiety_data.sample(frac=0.8,random_state=50)
anxiety_test=anxiety_data.drop(anxiety_train.index)
stress_train=stress_data.sample(frac=0.8,random_state=50)
stress_test=stress_data.drop(stress_train.index)

dprs_word2vecmodel=Word2Vec(depress_train["clean_text"].to_list(),size=100,window=3,min_count=2, workers=4,sg=1)
anxty_word2vecmodel=Word2Vec(anxiety_train["text"].to_list(),size=100,window=3,min_count=2, workers=4,sg=1)
strs_word2vecmodel=Word2Vec(stress_train["text"].to_list(),size=100,window=3,min_count=2, workers=4,sg=1)

#vectorization
def dprs_word2vectortest(tokenizedpost):
  vectors=[]
  if len(tokenizedpost)<1:
    return np.zeros(100)
  else:
    for word in tokenizedpost:
      if word in dprs_word2vecmodel.wv:
        vectors.append(dprs_word2vecmodel.wv.get_vector(word))
      else:
        vectors.append(np.random.rand(100))
  return np.mean(vectors,axis=0)          
def anx_word2vectortest(tokenizedpost):
  vectors=[]
  if len(tokenizedpost)<1:
    return np.zeros(100)
  else:
    for word in tokenizedpost:
      if word in anxty_word2vecmodel.wv:
        vectors.append(anxty_word2vecmodel.wv.get_vector(word))
      else:
        vectors.append(np.random.rand(100))
  return np.mean(vectors,axis=0)
def strs_word2vectortest(tokenizedpost):
  vectors=[]
  if len(tokenizedpost)<1:
    return np.zeros(100)
  else:
    for word in tokenizedpost:
      if word in strs_word2vecmodel.wv:
        vectors.append(strs_word2vecmodel.wv.get_vector(word))
      else:
        vectors.append(np.random.rand(100))
  return np.mean(vectors,axis=0)      
depress_test.dropna(inplace=True)
anxiety_test.dropna(inplace=True)
depress_train.dropna(inplace=True)
anxiety_train.dropna(inplace=True)
stress_train.dropna(inplace=True)
stress_test.dropna(inplace=True)
depress_test=depress_test[depress_test.astype(str)['clean_text'] != '[]']
anxiety_test=anxiety_test[anxiety_test.astype(str)['text'] != '[]']
depress_train=depress_train[depress_train.astype(str)['clean_text'] != '[]']
anxiety_train=anxiety_train[anxiety_train.astype(str)['text'] != '[]']
stress_train=stress_train[stress_train.astype(str)['text'] != '[]']
stress_test=stress_test[stress_test.astype(str)['text'] != '[]']
depress_test["vectors"]=depress_test["clean_text"].apply(lambda post:dprs_word2vectortest(post))
anxiety_test["vectors"]=anxiety_test["text"].apply(lambda post:anx_word2vectortest(post))
stress_test["vectors"]=stress_test["text"].apply(lambda post:strs_word2vectortest(post))
depress_train["vectors"]=depress_train["clean_text"].apply(lambda post:dprs_word2vectortest(post))
anxiety_train["vectors"]=anxiety_train["text"].apply(lambda post:anx_word2vectortest(post))
stress_train["vectors"]=stress_train["text"].apply(lambda post:strs_word2vectortest(post))
from numpy.linalg import norm
from sklearn.metrics import accuracy_score      
dep_rf=xgb.XGBClassifier(objective="binary:logistic", random_state=42)
y=np.array([np.array(post) for post in depress_train["vectors"]])
# print(y.shape)
deprf_model=dep_rf.fit(y,depress_train["is_depression"].values.ravel())
y=np.array([np.array(post) for post in depress_test["vectors"]])
dep_pred=deprf_model.predict(y)
# print(accuracy_score(depress_test["is_depression"],dep_pred))
def depress_rf(vector):
  depress_pred=deprf_model.predict(vector)
  return depress_pred
anx_rf=xgb.XGBClassifier(objective="binary:logistic", random_state=42)
y=np.array([np.array(post) for post in anxiety_train["vectors"]])
anxrf_model=anx_rf.fit(y,anxiety_train["label"].values.ravel())
y=np.array([np.array(post) for post in anxiety_test["vectors"]])
anx_pred=anxrf_model.predict(y)
# print(accuracy_score(anxiety_test["label"],anx_pred))
def anxiety_rf(vector):
  anxiety_pred=anxrf_model.predict(vector)
  return anxiety_pred
strs_rf=xgb.XGBClassifier(objective="binary:logistic", random_state=42)
y=np.array([np.array(post) for post in stress_train["vectors"]])
strsrf_model=strs_rf.fit(y,stress_train["label"].values.ravel())
y=np.array([np.array(post) for post in stress_test["vectors"]])
strs_pred=strsrf_model.predict(y)
# print(accuracy_score(stress_test["label"],strs_pred))
def stress_rf(vector):
  stress_pred=strsrf_model.predict(vector)
  return stress_pred
def depressedsimilar(vector):
  df1=depress_test[depress_test["is_depression"]==1]
  df2=depress_train[depress_train["is_depression"]==1]
  depressed=pd.concat([df1,df2])
  test=np.array(vector)
  templist=[]
  for postvector in depressed["vectors"]:
    pvarray=np.array(postvector)
    templist.append(np.dot(test,pvarray)/(norm(test)*norm(pvarray)))
  depressed['similarity']=templist
  depressed.sort_values(by="similarity",ascending=False,inplace=True)
  mostdepressed=depressed["similarity"].iloc[0]
  return mostdepressed
def anxioussimilar(vector):
  test=np.array(vector)
  df1=anxiety_test[anxiety_test["label"]==1]
  df2=anxiety_train[anxiety_train["label"]==1]
  anxious=pd.concat([df1,df2])
  templist=[]
  for postvector in anxious["vectors"]:
    pvarray=np.array(postvector)
    templist.append(np.dot(test,pvarray)/(norm(test)*norm(pvarray)))
  anxious['similarity']=templist
  anxious.sort_values(by="similarity",ascending=False,inplace=True)
  mostanxious=anxious["similarity"].iloc[0]
  return mostanxious
def stresssimilar(vector):
  df1=stress_test[stress_test["label"]==1]
  df2=stress_train[stress_train["label"]==1]
  stressed=pd.concat([df1,df2])
  test=np.array(vector)
  templist=[]
  for postvector in stressed["vectors"]:
    pvarray=np.array(postvector)
    templist.append(np.dot(test,pvarray)/(norm(test)*norm(pvarray)))
  stressed['similarity']=templist
  stressed.sort_values(by="similarity",ascending=False,inplace=True)
  moststressed=stressed["similarity"].iloc[0]
  return moststressed  
def DASlabel(query):

  text=re.sub("\'"," ",query)
  text=contractions(text)
  text=cleaning(text)
  post=re.sub(" +"," ",text)
  post=word_tokenize(post)
  stop_words=set(stopwords.words("english"))
  temp=[]
  for w in post:
    if w not in stop_words:
      temp.append(w)
  post=temp
  pos=[]
  pos=nltk.pos_tag(post)
  allowed_type=["JJ","RB","JJR","JJS","RBR","RBS","NN","NNS","NNP"]
  convert={"a":["JJ","JJR","JJS"],"r":["RB","RBR","RBS"],"n":["NN","NNS","NNP"]}
  cleaned_words=[]
  lematizer=WordNetLemmatizer();
  for w in pos:
    if w[1] in allowed_type:
      for key,val in convert.items():
        if w[1] in val:
          cleaned_words.append(lematizer.lemmatize(w[0],pos=key)) 
  post=list(set(cleaned_words))
  dvectorizedpost=dprs_word2vectortest(post)
  avectorizedpost=anx_word2vectortest(post)
  svectorizedpost=strs_word2vectortest(post)
  dv=np.array([np.array(dvectorizedpost)])
  av=np.array([np.array(avectorizedpost)])
  sv=np.array([np.array(svectorizedpost)])
  depressed,anxious,stressed=depress_rf(dv),anxiety_rf(av),stress_rf(sv)
  label=" "
  if(int(depressed)>int(anxious) and int(depressed)>int(stressed)):
    label="D"
  elif(int(anxious)>int(depressed) and int(anxious)>int(stressed)):
    label="A"
  elif(int(stressed)>int(depressed) and int(stressed)>int(anxious)):
    label="S"
  elif(int(anxious)==int(depressed) and int(depressed)==1 and int(stressed)==0):
    dv=depressedsimilar(dvectorizedpost)
    av=anxioussimilar(avectorizedpost)
    if(dv.all()>av.all()):
      label="D"
    else:
      label="A"
  elif(int(anxious)==int(stressed) and int(stressed)==1 and int(depressed)==0):
    sv=stresssimilar(svectorizedpost)
    av=anxioussimilar(avectorizedpost)
    if(sv.all()>av.all()):
      label="S"
    else:
      label="A"    
  elif(int(stressed)==int(depressed) and int(depressed)==1 and int(anxious)==0):
    dv=depressedsimilar(dvectorizedpost)
    sv=stresssimilar(svectorizedpost)
    if(dv.all()>sv.all()):
      label="D"
    else:
      label="S"
  else:
    dv=depressedsimilar(dvectorizedpost)
    sv=stresssimilar(svectorizedpost)
    av=anxioussimilar(avectorizedpost)
    if(dv.all()>sv.all() and dv.all()>av.all()):
      label="D"
    elif(sv.all()>dv.all() and sv.all()>av.all()):
      label="S"
    else:
      label="A"
  return label    

# #precisioncalculation
# def meanprecision(vector):
#   result=pd.DataFrame()
#   test=np.array(vector)
#   temp=[]
#   result['text']=depress_test["clean_text"]
#   result['vector']=depress_test["vectors"]
#   result['label']=depress_test["is_depression"]
#   for postvector in result["vector"]:
#     pvarray=np.array(postvector)
#     if (norm(test)*norm(pvarray))==0:
#       exit()
#     else:
#       temp.append(np.dot(test,pvarray)/(norm(test)*norm(pvarray)))
#   result['similarity']=temp
#   result.sort_values(by="similarity",ascending=False,inplace=True)
#   ranking=list(result["label"].head(10))
#   precision=[]
#   for i in range(1,11):
#     precision.append(np.sum(ranking[:i])/i)
#   if precision==[]:
#     return 0
#   return np.mean(precision)
# precision=[]
# for i,post in depress_test.iterrows():
#   if post["is_depression"]==0:
#     temp=meanprecision(post["vectors"])
#     precision.append(1-temp)
#   else:
#     precision.append(meanprecision(post["vectors"]))  





# # print("precision:",np.mean(precision))






<ipython-input-3-ba9bedb890f4>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anxiety_data['text']=anxiety_data['text'].apply(lambda post:re.sub(r"\'"," ",post))


In [ ]:
import whisper

def videototext(videopath):
  model1 = whisper.load_model("base")
  text = model1.transcribe(videopath,fp16=False,language="en")
  return text['text']


In [ ]:
import json
import werkzeug
from flask import Flask,request,jsonify
import numpy as np
import nltk
from pyngrok import ngrok

app=Flask(__name__)
port=5000
ngrok.set_auth_token("2LkQXIb6v3NfSP6gOta9IbhZHqt_819dkaTGBDo5EZ5ctxZ1Z")
public_url=ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> n http://127.0.0.1:{}\"".format(public_url,port))
app.config["BASE_URL"]=public_url
@app.route('/api',methods=['GET'])
def DAS_text():
    print("in DAS function")
    query=str(request.args['text'])
    label=DASlabel(query)
    d={}
    d['label']=label    
    return jsonify(d)

@app.route('/video',methods=['POST'])
def convertvideo():
  if(request.method=="POST"):
      
    videofile = request.files["video"]
    print("after videofile")
    filename = werkzeug.utils.secure_filename(videofile.filename)
        # print("\nReceived video File name : " + videofile.filename)
    videofile.save(filename)
    
    print("file saved!!!")
    text=videototext(filename)
    label=DASlabel(text)
    d={}
    d['label']=label
    response=jsonify(d)
    response.headers.add("Access-Control-Allow-Origin", "*")    

    return response

if __name__=="__main__":
    app.run()

 * ngrok tunnel "http://b646-35-194-78-113.ngrok.io" -> n http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
